#### Pre-processing

In [6]:
def load_images(path = '.', transform = None, batch_size = 32, shuffle = True, num_workers=5):
    '''
        Load images from a specific path
        
        Parameters
        ----------
        
        path: str
            Path to a folder of folders that contains images
        
        transform: torchvision.transforms
            Transformation function or a composition of transformation functions
        
        batch_size: int
            Amount of samples per batch
        
        shuffle: bool
            Data reshuffled at every epoch, if set to true
        
        num_workers: int
            Subprocesses to use for data loading, if zero run at the main process
            
        Usage
        -----
        
        >>> load_images(path='../data/train')
        
        Return
        ------
        
        Data Loader containing all images and its classes
        
        References
        ----------
        
        https://discuss.pytorch.org/t/how-to-load-png-using-dataloader/17079/2
        
        https://stats.stackexchange.com/questions/164876/tradeoff-batch-size-vs-number-of-iterations-to-train-a-neural-network
        
        https://pytorch.org/docs/stable/torchvision/transforms.html
        
        https://github.com/marvis/pytorch-yolo2/issues/89
        
        https://discuss.pytorch.org/t/how-can-i-load-a-number-of-images/12468/8
        
        Observations
        -----
        
        The order of the composition of the transformation functions matters. It's
        possible to iterate over the returned dataset by loading it and saving the
        result into two variables, e.g, `data, labels = load_images(path='../data/train')`
    '''
    
    from pathlib import Path
    
    pth = Path(path)
    
    # Try to start the jupyter lab at the root of the project
    if not pth.exists() or not pth.is_dir():
        raise Exception('Incompatible path or inexistent folder')
    
    from torchvision.datasets import ImageFolder
    
    from torchvision.transforms import Compose, ToTensor, Resize
    
    from torch.utils.data import DataLoader
    
    if not transform:
        transform = Compose([
            Resize((200,200)),  # (height, width) or an int 
            ToTensor()
        ])
    
    return DataLoader(
        dataset=ImageFolder(
            root=path,
            transform=transform
        ),
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=num_workers        
    )